<a href="https://colab.research.google.com/github/Kunal-Aher/Data/blob/main/Class_1_RDD_operations_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating RDDs

In [ ]:
#os.getenv('PYSPARK_SUBMIT_ARGS')

In [ ]:
import os

!export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)
from pyspark import SparkConf, SparkContext

#os.environ['PYSPARK_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3'
#os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/usr/local/Cellar/python@3.10/3.10.9/bin/python3'
#os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/local/bin/python3'

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("rdd_demo").getOrCreate()
#spark = SparkSession.builder.master("local[1]").appName("rdd_demo").getOrCreate()
#spark = SparkSession.builder.master("local").config("spark.driver.bindAddress", "127.0.0.1").appName("rdd_demo").getOrCreate()
#spark = SparkSession.builder.master("local").config("spark.ui.port", "4041").appName("rdd_demo").getOrCreate()
#spark = SparkSession.builder.master("local").config("spark.ui.reverseProxy", "true").appName("rdd_demo").getOrCreate()


#sc.stop()
conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext.getOrCreate()

24/12/23 10:22:18 WARN Utils: Your hostname, Akashs-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.33.206 instead (on interface en0)
24/12/23 10:22:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 10:22:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark

In [ ]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "false")\
  .option("header", "true")\
  .csv("/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv")

In [ ]:
flightData2015.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count='15'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count='1'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count='344'),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count='15'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count='62')]

In [ ]:
flightData2015

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string]

In [ ]:
type(flightData2015)

pyspark.sql.dataframe.DataFrame

In [ ]:
##### read using datasource API
flightData2015 = spark\
  .read\
  .option("inferSchema", "false")\
  .option("header", "true")\
  .csv("/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv")

# COMMAND ----------
flightData2015=flightData2015.toDF("dest","source","count").rdd
print(type(flightData2015))
print(flightData2015.take(1))
print(flightData2015.count())

<class 'pyspark.rdd.RDD'>


[Row(dest='United States', source='Romania', count='15')]
256


In [ ]:
##### read using sparkcontext
spth="/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv"
sc_flightData2015=spark.sparkContext.textFile(spth)
print(type(sc_flightData2015))
print(sc_flightData2015.take(2))

<class 'pyspark.rdd.RDD'>
['DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count', 'United States,Romania,15']


In [ ]:
##### convert pandas file to RDD
import pandas as pd

spth="/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv"
pd_flightData2015=pd.read_csv(spth, header=0)
print(type(pd_flightData2015))
print(pd_flightData2015.head())
pd_flightData2015=spark.createDataFrame(pd_flightData2015).rdd
print(type(pd_flightData2015))
print(pd_flightData2015.take(1))

<class 'pandas.core.frame.DataFrame'>
  DEST_COUNTRY_NAME ORIGIN_COUNTRY_NAME  count
0     United States             Romania     15
1     United States             Croatia      1
2     United States             Ireland    344
3             Egypt       United States     15
4     United States               India     62


/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


<class 'pyspark.rdd.RDD'>
[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)]


In [ ]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)
words.take(10)

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [ ]:
##### from a collection of text
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

words.setName("myWords")
print(words.name()) # myWords
print(type(words))
print(words.take(5))
words.getNumPartitions()

myWords
<class 'pyspark.rdd.RDD'>
['Spark', 'The', 'Definitive', 'Guide', ':']


2

In [ ]:
### from a range of numbers
myRange = spark.range(1000000000000).toDF("number").rdd.map(lambda row: row[0])
#myRange = spark.range(1000).toDF("number")
print(myRange.take(5))
print(type(myRange))
myRange.getNumPartitions()

[0, 1, 2, 3, 4]
<class 'pyspark.rdd.PipelinedRDD'>


1

# filtering

In [ ]:
def parseLine(line):
    fields = line.split(',')
    date = fields[0]
    p_open = fields[1]
    p_close = fields[5]
    return (date, p_open, p_close)

In [ ]:
spth="/Users/aakash/Downloads/spark-code/data/RELIANCE.csv"
sdt=spark.sparkContext.textFile(spth)
sdt=sdt.map(parseLine)
sdt.take(2)

[('Date', 'Open', 'Close'), ('2022-01-01', '100', '10000')]

In [ ]:
spth="/Users/aakash/Downloads/spark-code/data/RELIANCE.csv"
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true") \
    .load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd \
    .map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
print(type(o_sdt))
print(o_sdt.take(2))

o_sdt=o_sdt.filter(lambda row: row[2] > row[1])
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())

5
<class 'pyspark.rdd.PipelinedRDD'>
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000)]
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000), (datetime.date(2022, 1, 3), 300, 30000), (datetime.date(2022, 1, 4), 400, 40000), (datetime.date(2022, 1, 5), 500, 50000)]
<class 'pyspark.rdd.PipelinedRDD'>
5


## filter function

In [ ]:
def HighClose(row):
    ## Discuss
    if row[2] > row[1]:
        return(row)

In [ ]:
o_sdt = spark.read.format("CSV").option("header","true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
o_sdt=o_sdt.filter(lambda row: HighClose(row))
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())

5
[('2022-01-01', '100', '10000'), ('2022-01-02', '200', '20000'), ('2022-01-03', '300', '30000'), ('2022-01-04', '400', '40000'), ('2022-01-05', '500', '50000')]
<class 'pyspark.rdd.PipelinedRDD'>
5


In [ ]:
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
o_sdt=o_sdt.filter(lambda row: HighClose(row))
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())


# Maps

In [ ]:
def to_to_mill(row):
    return (row[0], row[1], row[2], round(row[3],0))

In [ ]:
spth="/Users/aakash/Downloads/spark-code/data/RELIANCE.csv"
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5], row[7]))
print(o_sdt.take(2))
o_sdt=o_sdt.map(to_to_mill)
print(type(o_sdt))

[(datetime.date(2022, 1, 1), 100, 10000, None), (datetime.date(2022, 1, 2), 200, 20000, None)]
<class 'pyspark.rdd.PipelinedRDD'>


# flatMap

In [ ]:
def Func(lines):
    lines = lines.lower()
    lines = lines.split(" ")
    return lines

#sc.stop()
#conf = SparkConf().setMaster("local").setAppName("wordcount")
#sc = SparkContext.getOrCreate()

spth="/Users/aakash/Downloads/spark-code/data/sherlock_holmes.txt"
input_file = sc.textFile(spth)

In [ ]:
print(input_file.take(1))

['A SCANDAL IN BOHEMIA']


In [ ]:
rdd1 = input_file.flatMap(Func)

In [ ]:
rdd1.take(5)

['a', 'scandal', 'in', 'bohemia', 'i.']

In [ ]:
rdd1.map(lambda x: (x,1)).take(5)

[('a', 1), ('scandal', 1), ('in', 1), ('bohemia', 1), ('i.', 1)]

In [ ]:
rdd2=rdd1 \
.map(lambda x: (x,1)) \
.groupByKey() \
.mapValues(sum) \
.map(lambda x: (x[1],x[0])) \
.sortByKey(False)

rdd2.take(5)

[(556, 'the'), (321, ''), (305, 'and'), (302, 'of'), (299, 'to')]

## Reduce

In [ ]:
spark.sparkContext.parallelize(range(1,5)).reduce(lambda x, y: x+y)

10

In [ ]:
Reduce works on 2 consecutive inputs
and performs incremental operations on them

Input: [1,2,3,4]
Reduce step 1 -
    Input: 1,2
    Output: 1+2

Reduce step 2 -
    Input: 3
    Output: 3+3= 6

Reduce step 3 -
    Input: 4
    Output: 6+4 = 10